In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

device=('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4

train_set=datasets.MNIST(root='./data',train=True,download=True,transform=ToTensor())
test_set=datasets.MNIST(root='./data',train=False,download=True,transform=ToTensor())

train_loader=DataLoader(train_set,shuffle=True,batch_size=batch_size)
test_loader=DataLoader(test_set,shuffle=False,batch_size=batch_size)

class CNNMnist(nn.Module):
    def __init__(self):
        super(CNNMnist,self).__init__()
        self.net=nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride=2),#13x13x128
            nn.Conv2d(64,128,kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride=2),#5x5x128
            nn.Conv2d(128,64,kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride=2)
        )
        self.classification_head=nn.Sequential(
            nn.Linear(64,20,bias=True),
            nn.Sigmoid(),
            nn.Linear(20,10,bias=True)
        )
    def forward(self,x):
        features=self.net(x)
        x=self.classification_head(features.view(batch_size,-1))
        return x

model=CNNMnist().to(device)
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)
loss_fn=nn.CrossEntropyLoss()

def train_one_epoch():
    total_loss=0
    for i,data in enumerate(train_loader):
        input,label=data
        input,label=input.to(device),label.to(device)
        optimizer.zero_grad()
        output=model(input)
        loss=loss_fn(output,label)
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    return total_loss/len(train_loader)

epochs=10

for i in range(epochs):
    model.train(True)
    res=train_one_epoch()
    print('epoch= ',i+1,' error: ',res)



100%|██████████| 9912422/9912422 [00:00<00:00, 286568959.71it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 23130741.61it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 118742018.73it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5870733.06it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

epoch=  1  error:  2.2982987076441446
epoch=  2  error:  2.2229804660161334
epoch=  3  error:  1.4355088044385116
epoch=  4  error:  0.6730265868614117
epoch=  5  error:  0.4050095774662991
epoch=  6  error:  0.2964132331351439
epoch=  7  error:  0.2383039150017003
epoch=  8  error:  0.20221605622594555
epoch=  9  error:  0.17617948060197136
epoch=  10  error:  0.15748407102115453


In [ ]:
success=0
count=0
for i,(input,label) in enumerate(test_loader):
  input=input.to(device)
  label=label.to(device)
  output=model(input)
  _,predicted=torch.max(output,1)
  count+=label.size(0)
  success+=(predicted==label).sum()
print(success/count*100)

tensor(96.5000, device='cuda:0')


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

device=('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=4

train_set=datasets.MNIST(root='./data',transform=ToTensor(),train=True,download=True)
test_set=datasets.MNIST(root='./data',transform=ToTensor(),train=False,download=True)


train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_set,batch_size=batch_size,shuffle=False)

class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel,self).__init__()
    self.net=nn.Sequential(
        nn.Conv2d(1,64,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2),
        nn.Conv2d(64,128,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2),
        nn.Conv2d(128,64,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d((2,2),stride=2)
    )
    self.classification_head=nn.Sequential(
        nn.Linear(64,20,bias=True),
        nn.Sigmoid(),
        nn.Linear(20,10,bias=True)
    )
  def forward(self,x):
    feature=self.net(x)
    x=self.classification_head(feature.view(batch_size,-1))
    return x


model=CNNModel().to(device)
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)
loss_fn=nn.CrossEntropyLoss()

def train_one_epoch():
  total_loss=0
  for i,(input,label) in enumerate(train_loader):
    input=input.to(device)
    label=label.to(device)
    optimizer.zero_grad()
    output=model(input)
    loss=loss_fn(output,label)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()
  return total_loss/len(train_loader)

for i in range(10):
  model.train()
  res=train_one_epoch()
  model.eval()
  print(res)


2.2980764725208283
2.231238443684578
1.4751570805410543
0.7578671388606231
0.4664615522533655
0.3344663819178939
0.2636742242902517
0.2202937032810102
0.19076396482313673
0.1676775827517112


In [ ]:
count=0
correct=0
for i,(input,label) in enumerate(test_loader):
  input=input.to(device)
  label=label.to(device)
  output=model(input)
  _,predicted=torch.max(output,1)
  count+=label.size(0)
  correct+=(predicted==label).sum()

print(correct/count*100)


print([p.numel() for p in model.parameters() if p.requires_grad])


tensor(96.4000, device='cuda:0')
[576, 64, 73728, 128, 73728, 64, 1280, 20, 200, 10]


Overall Team Accuracy (Weighted MAPE): 6684067.15%
